In [1]:
import pyspark as ps 
spark = ps.sql.SparkSession.builder.master("local").appName("firstmodel").getOrCreate()

In [2]:
df= spark.read.csv('RegressionData_1.csv',header=True,inferSchema=True)
df

DataFrame[year_target: int, Uid: int, age: int, Pid: int, city: string, state: string, subscribe: int, MARRIAGE: int, EDUCATION: int]

In [3]:
df.printSchema()
df.columns

root
 |-- year_target: integer (nullable = true)
 |-- Uid: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- Pid: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- subscribe: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)



['year_target',
 'Uid',
 'age',
 'Pid',
 'city',
 'state',
 'subscribe',
 'MARRIAGE',
 'EDUCATION']

In [4]:
df.show(5)

+-----------+----+---+------+---------+-----+---------+--------+---------+
|year_target| Uid|age|   Pid|     city|state|subscribe|MARRIAGE|EDUCATION|
+-----------+----+---+------+---------+-----+---------+--------+---------+
|       1970|9052| 33|229536|     agra|   HP|        1|       1|        3|
|       1944|9762| 25|407967|  gurgaon|   MP|        0|       2|        2|
|       1947|5595| 30| 75478|bangalore|   MP|        1|       1|        1|
|       2004|5212| 31|872871|bangalore|   MP|        0|       2|        2|
|       1986|1064| 40| 57864|    delhi|   AP|        0|       1|        2|
+-----------+----+---+------+---------+-----+---------+--------+---------+
only showing top 5 rows



In [7]:
df=df.drop('city','state')
df.show(5)

+-----------+----+---+------+---------+--------+---------+
|year_target| Uid|age|   Pid|subscribe|MARRIAGE|EDUCATION|
+-----------+----+---+------+---------+--------+---------+
|       1970|9052| 33|229536|        1|       1|        3|
|       1944|9762| 25|407967|        0|       2|        2|
|       1947|5595| 30| 75478|        1|       1|        1|
|       2004|5212| 31|872871|        0|       2|        2|
|       1986|1064| 40| 57864|        0|       1|        2|
+-----------+----+---+------+---------+--------+---------+
only showing top 5 rows



In [18]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

In [9]:
assemblerInputs = ['year_target','Uid','age','Pid','MARRIAGE','EDUCATION']
vector_assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
assembler_temp=vector_assembler.transform(df)
assembler_temp.show(5)

+-----------+----+---+------+---------+--------+---------+--------------------+
|year_target| Uid|age|   Pid|subscribe|MARRIAGE|EDUCATION|            features|
+-----------+----+---+------+---------+--------+---------+--------------------+
|       1970|9052| 33|229536|        1|       1|        3|[1970.0,9052.0,33...|
|       1944|9762| 25|407967|        0|       2|        2|[1944.0,9762.0,25...|
|       1947|5595| 30| 75478|        1|       1|        1|[1947.0,5595.0,30...|
|       2004|5212| 31|872871|        0|       2|        2|[2004.0,5212.0,31...|
|       1986|1064| 40| 57864|        0|       1|        2|[1986.0,1064.0,40...|
+-----------+----+---+------+---------+--------+---------+--------------------+
only showing top 5 rows



In [13]:
assembler=assembler_temp.drop('year_target','Uid','age','Pid','MARRIAGE','EDUCATION')
assembler.show(5)

+---------+--------------------+
|subscribe|            features|
+---------+--------------------+
|        1|[1970.0,9052.0,33...|
|        0|[1944.0,9762.0,25...|
|        1|[1947.0,5595.0,30...|
|        0|[2004.0,5212.0,31...|
|        0|[1986.0,1064.0,40...|
+---------+--------------------+
only showing top 5 rows



In [17]:
assembler.select("subscribe").distinct().count()

2

In [21]:
label_indexer = StringIndexer(inputCol="subscribe", outputCol="labelIndex")
ml_dataset = label_indexer.fit(assembler).transform(assembler)
ml_dataset.show(5)

+---------+--------------------+----------+
|subscribe|            features|labelIndex|
+---------+--------------------+----------+
|        1|[1970.0,9052.0,33...|       1.0|
|        0|[1944.0,9762.0,25...|       0.0|
|        1|[1947.0,5595.0,30...|       1.0|
|        0|[2004.0,5212.0,31...|       0.0|
|        0|[1986.0,1064.0,40...|       0.0|
+---------+--------------------+----------+
only showing top 5 rows



In [91]:
(trainDataset, testDataset) = ml_dataset.randomSplit([.7,0.3])

In [62]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [92]:
dt_model = DecisionTreeClassifier(labelCol="labelIndex",featuresCol="features")
model = dt_model.fit(trainDataset)

Py4JJavaError: An error occurred while calling o2728.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:105)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1090)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:414)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1088)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopFile$4(PairRDDFunctions.scala:1061)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withSc

In [93]:
predictions = model.transform(testDataset)
predictions.show(5)

+---------+--------------------+----------+---------------+--------------------+----------+
|subscribe|            features|labelIndex|  rawPrediction|         probability|prediction|
+---------+--------------------+----------+---------------+--------------------+----------+
|        0|[1920.0,1407.0,33...|       0.0|[1184.0,1200.0]|[0.49664429530201...|       1.0|
|        0|[1920.0,1853.0,43...|       0.0|  [267.0,304.0]|[0.46760070052539...|       1.0|
|        0|[1920.0,1986.0,78...|       0.0|[1184.0,1200.0]|[0.49664429530201...|       1.0|
|        0|[1920.0,2079.0,34...|       0.0|[1184.0,1200.0]|[0.49664429530201...|       1.0|
|        0|[1920.0,2407.0,68...|       0.0|    [11.0,10.0]|[0.52380952380952...|       0.0|
+---------+--------------------+----------+---------------+--------------------+----------+
only showing top 5 rows



In [94]:
evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex",predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy_dtc :",accuracy)

Accuracy_dtc : 0.508702909505391


In [95]:
from pyspark.ml.classification import RandomForestClassifier
rf_model = RandomForestClassifier(labelCol="labelIndex",featuresCol="features",numTrees=20)
model2 = rf_model.fit(trainDataset)

In [104]:
testDataset=testDataset.drop('subscribe')

In [105]:
predictions2 = model2.transform(testDataset)
predictions2.show(5)

+--------------------+----------+--------------------+--------------------+----------+
|            features|labelIndex|       rawPrediction|         probability|prediction|
+--------------------+----------+--------------------+--------------------+----------+
|[1920.0,1407.0,33...|       0.0|[10.3651790074536...|[0.51825895037268...|       0.0|
|[1920.0,1853.0,43...|       0.0|[10.4263504181698...|[0.52131752090849...|       0.0|
|[1920.0,1986.0,78...|       0.0|[10.2787213590414...|[0.51393606795207...|       0.0|
|[1920.0,2079.0,34...|       0.0|[10.3651790074536...|[0.51825895037268...|       0.0|
|[1920.0,2407.0,68...|       0.0|[9.85040241902918...|[0.49252012095145...|       1.0|
+--------------------+----------+--------------------+--------------------+----------+
only showing top 5 rows



In [71]:
accuracy_nt__dtc_train=0.49141385231825985
accuracy_nt_dtc_test=0.503819329126536

In [72]:
difference_accuracy = accuracy_nt_dtc_test - accuracy
difference_accuracy

0.03236910887024891

In [90]:
0.5096597306725575-accuracy_nt_dtc_test

0.005840401546021501